In [1]:
import numpy as np

from core.TrajectoryDF import NumPandasTraj as NumTrajDF
from features.spatial_features import SpatialFeatures as spatial
from utilities.conversions import Conversions as con
import pandas as pd
import time
np.seterr(invalid='ignore')
start = time.time()

In [2]:
%%time
# Reading the geolife dataset and converting to NumPandasTraj.
geolife = pd.read_csv('./data/geolife_sample.csv')
geolife = NumTrajDF(geolife,'lat','lon','datetime','id')

# Reading the gulls dataset and converting to NumPandasTraj.
gulls = pd.read_csv('./data/gulls.csv')
gulls = NumTrajDF(gulls,
                 latitude='location-lat',
                 longitude='location-long',
                 datetime='timestamp',
                 traj_id='tag-local-identifier',
                 rest_of_columns=[])

# Reading the atlantic dataset, cleaning it up and then converting
# it to NumPandasTraj.
atlantic = pd.read_csv('./data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic = NumTrajDF(atlantic,
                         latitude='Latitude',
                         longitude='Longitude',
                         datetime='DateTime',
                         traj_id='ID',
                         rest_of_columns=[])
atlantic.head()

CPU times: user 8.42 s, sys: 103 ms, total: 8.53 s
Wall time: 8.53 s


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-06-25 00:00:00,AL011851,UNNAMED,18510625,0,,HU,28.0,-94.8,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 06:00:00,AL011851,UNNAMED,18510625,600,,HU,28.0,-95.4,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 12:00:00,AL011851,UNNAMED,18510625,1200,,HU,28.0,-96.0,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 18:00:00,AL011851,UNNAMED,18510625,1800,,HU,28.1,-96.5,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1851-06-25 21:00:00,AL011851,UNNAMED,18510625,2100,L,HU,28.2,-96.8,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [3]:
%%time

# Now, lets get the bounding boxes of all the dataframes.
print(spatial.get_bounding_box(geolife))
print(spatial.get_bounding_box(gulls))
print(spatial.get_bounding_box(atlantic))

(22.147577, 113.54884299999999, 41.132062, 121.156224)
(-62.71617, 4.02517, 65.2325, 103.19317)
(7.2, -109.5, 81.0, 63.0)
CPU times: user 8.1 ms, sys: 8 µs, total: 8.11 ms
Wall time: 6.61 ms


In [4]:
%%time

# Now, lets get the start times of all the dataframes without
# giving a traj_id.
print(spatial.get_start_location(geolife))
print(spatial.get_start_location(gulls))
print(spatial.get_start_location(atlantic))

               lat         lon
traj_id                       
1        39.984224  116.319402
5        40.004155  116.321337
              lat       lon
traj_id                    
91785    64.98267  35.74300
91797    64.96767  35.73650
91769    54.18433   7.91283
91843    55.42533  49.34817
91913    64.97850  35.72667
...           ...       ...
91814    55.43233  49.34283
91763    62.78917  28.76983
91823    54.18600   7.91867
91752    62.40617  29.46983
91916    54.18533   7.91500

[126 rows x 2 columns]
           lat   lon
traj_id             
AL091869  17.0 -67.5
AL041862  13.0 -61.0
AL011870  30.5 -88.0
AL071861  35.3 -75.3
AL071853  15.0 -37.0
...        ...   ...
AL041926  13.8 -41.2
AL142012  15.5 -38.0
AL201969  16.0 -47.0
AL141971  25.5 -71.5
AL031899  11.7 -31.0

[1814 rows x 2 columns]
CPU times: user 591 ms, sys: 373 ms, total: 963 ms
Wall time: 5.11 s


In [5]:
%%time

# Now lets check the start times of all dataframes with a traj_id.
print(spatial.get_start_location(geolife, '1'))
print(spatial.get_start_location(gulls, "91732"))
print(spatial.get_start_location(atlantic, 'AL011851'))

(39.984224, 116.319402)
(61.24783000000001, 24.586170000000003)
(28.0, -94.8)
CPU times: user 130 ms, sys: 244 µs, total: 131 ms
Wall time: 129 ms


In [6]:
%%time

# Now, lets get the start times of all the dataframes without
# giving a traj_id.
print(spatial.get_end_location(geolife))
print(spatial.get_end_location(gulls))
print(spatial.get_end_location(atlantic))

               lat         lon
traj_id                       
1        39.977879  116.326628
5        39.999978  116.327460
              lat       lon
traj_id                    
91785    64.98267  35.74300
91797    64.96767  35.73633
91769    54.18417   7.91350
91843    55.43533  49.36383
91913    64.96700  35.74083
...           ...       ...
91814    -1.79117  32.80583
91763     6.97833  29.51200
91823    61.31100  24.57767
91752     8.06067  38.85033
91916    56.56600  21.19633

[126 rows x 2 columns]
           lat   lon
traj_id             
AL091869  17.0 -67.5
AL041862  13.0 -61.0
AL011870  30.5 -88.0
AL071861  35.3 -75.3
AL071853  15.0 -37.0
...        ...   ...
AL041926  58.2 -49.2
AL142012  37.6 -29.1
AL201969  26.1 -61.6
AL141971  35.7 -69.9
AL031899  49.0 -15.5

[1814 rows x 2 columns]
CPU times: user 549 ms, sys: 412 ms, total: 961 ms
Wall time: 4.84 s


In [7]:
%%time

# Now lets check the start times of all dataframes with a traj_id.
print(spatial.get_end_location(geolife, '1'))
print(spatial.get_end_location(gulls, "91732"))
print(spatial.get_end_location(atlantic, 'AL011851'))

(39.977878999999994, 116.326628)
(31.09783, 29.812170000000002)
(31.0, -100.2)
CPU times: user 126 ms, sys: 11.6 ms, total: 137 ms
Wall time: 136 ms


In [8]:
%%time

# Now, lets calculate the distance between 2 consecutive points for
# each dataset one by one.
# First, we will check the distances between 2 consecutive points
# in the geolife dataset.
geolife = spatial.create_distance_between_consecutive_column(geolife)
geolife.head(10)

CPU times: user 351 ms, sys: 44.4 ms, total: 395 ms
Wall time: 1.12 s


,,lat,lon,Distance_prev_to_curr
DateTime,traj_id,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671
2008-10-23 05:53:23,1,39.984710,116.319865,66.555997
2008-10-23 05:53:28,1,39.984674,116.319810,6.162987
2008-10-23 05:53:33,1,39.984623,116.319773,6.488225
2008-10-23 05:53:38,1,39.984606,116.319732,3.971848


In [9]:
%%time

# # Here, we check the distance between 2 consecutive points
# # in the seagulls dataset.
gulls = spatial.create_distance_between_consecutive_column(gulls)
gulls.head()

CPU times: user 240 ms, sys: 44.6 ms, total: 285 ms
Wall time: 732 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Distance_prev_to_curr
DateTime,traj_id,,,,,,,,,
2009-08-15 15:00:00,91733,1082636186,True,7.91883,54.18600,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,NaN
2009-08-15 21:00:00,91733,1082636187,True,7.91917,54.18567,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,42.847231
2009-08-16 06:00:00,91733,1082636188,True,7.91500,54.18533,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,273.951247
2009-08-16 09:00:00,91733,1082636189,True,7.91917,54.18583,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,276.967082
2009-08-16 15:00:00,91733,1082636190,True,8.64117,54.33383,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,49697.545831


In [10]:
%%time

# Here, we check the distance between 2 consecutive points
# in the atlantic dataset.
atlantic = spatial.create_distance_between_consecutive_column(atlantic)
atlantic.head()

CPU times: user 228 ms, sys: 135 ms, total: 362 ms
Wall time: 398 ms


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Distance_prev_to_curr
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-07-05 12:00:00,AL021851,UNNAMED,18510705,1200,,HU,22.2,-97.6,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NaN
1851-07-10 12:00:00,AL031851,UNNAMED,18510710,1200,,TS,12.0,-60.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NaN
1853-08-05 12:00:00,AL011853,UNNAMED,18530805,1200,,TS,32.5,-69.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NaN
1853-08-10 12:00:00,AL021853,UNNAMED,18530810,1200,,TS,12.0,-60.0,40,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NaN
1853-09-21 12:00:00,AL051853,UNNAMED,18530921,1200,,TS,20.0,-95.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,NaN


In [11]:
%%time

# Now, lets calculate the distance from start to current point for
# each dataset one by one.
# Here, we check the distance between start to all points
# in the geolife dataset.

geolife = spatial.create_distance_from_start_column(geolife)
geolife.head()

CPU times: user 254 ms, sys: 92.2 ms, total: 346 ms
Wall time: 836 ms


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr
DateTime,traj_id,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073


In [12]:
%%time

# Here, we check the distance between start to all points
# in the seagulls dataset.
gulls = spatial.create_distance_from_start_column(gulls)
gulls.head()

CPU times: user 158 ms, sys: 56.2 ms, total: 214 ms
Wall time: 529 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Distance_prev_to_curr,Distance_start_to_curr
DateTime,traj_id,,,,,,,,,,
2009-08-15 15:00:00,91733,1082636186,True,7.91883,54.18600,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,NaN,NaN
2009-08-15 21:00:00,91733,1082636187,True,7.91917,54.18567,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,42.847231,42.847231
2009-08-16 06:00:00,91733,1082636188,True,7.91500,54.18533,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,273.951247,260.103940
2009-08-16 09:00:00,91733,1082636189,True,7.91917,54.18583,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,276.967082,29.098770
2009-08-16 15:00:00,91733,1082636190,True,8.64117,54.33383,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,49697.545831,49712.038359


In [13]:
%%time

# Here, we check the distance between start to all points
# in the atlantic dataset.
atlantic = spatial.create_distance_from_start_column(atlantic)
atlantic.head()

CPU times: user 230 ms, sys: 142 ms, total: 371 ms
Wall time: 398 ms


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Distance_prev_to_curr,Distance_start_to_curr
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-07-05 12:00:00,AL021851,UNNAMED,18510705,1200,,HU,22.2,-97.6,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,NaN,NaN
1851-07-10 12:00:00,AL031851,UNNAMED,18510710,1200,,TS,12.0,-60.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,NaN,NaN
1853-08-05 12:00:00,AL011853,UNNAMED,18530805,1200,,TS,32.5,-69.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,NaN,NaN
1853-08-10 12:00:00,AL021853,UNNAMED,18530810,1200,,TS,12.0,-60.0,40,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,NaN,NaN
1853-09-21 12:00:00,AL051853,UNNAMED,18530921,1200,,TS,20.0,-95.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,NaN,NaN


In [14]:
%%time

# Here, we are calculating the distance travelled in traj_id 1
# on the date 2008-10-23.
spatial.get_distance_travelled_by_date_and_traj_id(geolife,
                                                   '2008-10-23',
                                                   '1')

CPU times: user 131 ms, sys: 15.2 ms, total: 146 ms
Wall time: 145 ms


14748.27854559045

In [15]:
%%time

# Now, lets check the distance of all the points
# in the geolife dataset from the coordinates (0, 0).
geolife = spatial.create_distance_from_given_point_column(geolife,
                                                          (0, 0))
geolife.head()

CPU times: user 259 ms, sys: 77.1 ms, total: 336 ms
Wall time: 807 ms


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr,"Distance_to_(0, 0)"
DateTime,traj_id,,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN,1.221587e+07
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153,1.221587e+07
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428,1.221588e+07
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895,1.221588e+07
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073,1.221588e+07


In [16]:
%%time

# Now, lets check if all the points in the geolife dataset
# are within 12000 KMs of the coordinates (0, 0).
geolife = spatial.create_point_within_range_column(geolife,
                                                   (0,0),
                                                   15000000)
geolife.head()

CPU times: user 390 ms, sys: 87.1 ms, total: 477 ms
Wall time: 949 ms


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr,"Distance_to_(0, 0)","Within_15000000_m_from_(0, 0)"
DateTime,traj_id,,,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN,1.221587e+07,True
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153,1.221587e+07,True
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428,1.221588e+07,True
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895,1.221588e+07,True
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073,1.221588e+07,True


In [17]:
%%time

# Now, lets calculate the speed of the object between 2
# consecutive trajectory points.
# First, we are doing in the geolife dataset.
geolife = spatial.create_speed_from_prev_column(geolife)
geolife.head()

CPU times: user 32 ms, sys: 3.33 ms, total: 35.3 ms
Wall time: 33.9 ms


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr,"Distance_to_(0, 0)","Within_15000000_m_from_(0, 0)",Speed_prev_to_curr
DateTime,traj_id,,,,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN,1.221587e+07,True,NaN
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153,1.221587e+07,True,13.690153
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428,1.221588e+07,True,0.000158
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895,1.221588e+07,True,0.364217
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073,1.221588e+07,True,0.577934


In [18]:
%%time

# Now lets calculate the speed between 2 consecutive
# points in the atlantic dataset.
atlantic = spatial.create_speed_from_prev_column(atlantic)
atlantic.head()

CPU times: user 17 ms, sys: 170 µs, total: 17.2 ms
Wall time: 15.9 ms


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-07-05 12:00:00,AL021851,UNNAMED,18510705,1200,,HU,22.2,-97.6,80,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN
1851-07-10 12:00:00,AL031851,UNNAMED,18510710,1200,,TS,12.0,-60.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN
1853-08-05 12:00:00,AL011853,UNNAMED,18530805,1200,,TS,32.5,-69.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN
1853-08-10 12:00:00,AL021853,UNNAMED,18530810,1200,,TS,12.0,-60.0,40,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN
1853-09-21 12:00:00,AL051853,UNNAMED,18530921,1200,,TS,20.0,-95.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN


In [19]:
%%time

# Now lets calculate the speed between 2 consecutive
# points in the seagulls dataset.
gulls = spatial.create_speed_from_prev_column(gulls)
gulls.head()

CPU times: user 24.7 ms, sys: 109 µs, total: 24.8 ms
Wall time: 23 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr
DateTime,traj_id,,,,,,,,,,,
2009-08-15 15:00:00,91733,1082636186,True,7.91883,54.18600,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,NaN,NaN,NaN
2009-08-15 21:00:00,91733,1082636187,True,7.91917,54.18567,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,42.847231,42.847231,0.001984
2009-08-16 06:00:00,91733,1082636188,True,7.91500,54.18533,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,273.951247,260.103940,0.008455
2009-08-16 09:00:00,91733,1082636189,True,7.91917,54.18583,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,276.967082,29.098770,0.025645
2009-08-16 15:00:00,91733,1082636190,True,8.64117,54.33383,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,49697.545831,49712.038359,2.300812


In [20]:
%%time

# Now lets check the acceleration of the object
# between 2 consecutive points.
# First, lets check for the geolife dataset.
geolife = spatial.create_acceleration_from_prev_column(geolife)
geolife.head()

CPU times: user 34.7 ms, sys: 439 µs, total: 35.1 ms
Wall time: 33.3 ms


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr,"Distance_to_(0, 0)","Within_15000000_m_from_(0, 0)",Speed_prev_to_curr,Acceleration_prev_to_curr
DateTime,traj_id,,,,,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN,1.221587e+07,True,NaN,NaN
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153,1.221587e+07,True,13.690153,NaN
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428,1.221588e+07,True,0.000158,-0.000292
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895,1.221588e+07,True,0.364217,0.072812
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073,1.221588e+07,True,0.577934,0.042744


In [21]:
%%time

# Now lets check the acceleration of the gulls dataset.
gulls = spatial.create_acceleration_from_prev_column(gulls)
gulls.head()

CPU times: user 25.8 ms, sys: 299 µs, total: 26.1 ms
Wall time: 24.1 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr
DateTime,traj_id,,,,,,,,,,,,
2009-08-15 15:00:00,91733,1082636186,True,7.91883,54.18600,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,NaN,NaN,NaN,NaN
2009-08-15 21:00:00,91733,1082636187,True,7.91917,54.18567,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,42.847231,42.847231,0.001984,NaN
2009-08-16 06:00:00,91733,1082636188,True,7.91500,54.18533,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,273.951247,260.103940,0.008455,1.997413e-07
2009-08-16 09:00:00,91733,1082636189,True,7.91917,54.18583,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,276.967082,29.098770,0.025645,1.591650e-06
2009-08-16 15:00:00,91733,1082636190,True,8.64117,54.33383,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,49697.545831,49712.038359,2.300812,1.053318e-04


In [22]:
%%time

# Now lets check the acceleration of the atlantic dataset.
atlantic = spatial.create_acceleration_from_prev_column(atlantic)
atlantic.head()

CPU times: user 12.7 ms, sys: 3.45 ms, total: 16.1 ms
Wall time: 14.9 ms


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-07-05 12:00:00,AL021851,UNNAMED,18510705,1200,,HU,22.2,-97.6,80,-999,-999,...,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN
1851-07-10 12:00:00,AL031851,UNNAMED,18510710,1200,,TS,12.0,-60.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN
1853-08-05 12:00:00,AL011853,UNNAMED,18530805,1200,,TS,32.5,-69.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN
1853-08-10 12:00:00,AL021853,UNNAMED,18530810,1200,,TS,12.0,-60.0,40,-999,-999,...,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN
1853-09-21 12:00:00,AL051853,UNNAMED,18530921,1200,,TS,20.0,-95.0,50,-999,-999,...,-999,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN


In [23]:
%%time

# Now lets check the jerk of the object between 2 consecutive
# points.
# First, we will do it for the geolife dataset.
geolife = spatial.create_jerk_from_prev_column(geolife)
geolife.head()

CPU times: user 38.9 ms, sys: 473 µs, total: 39.3 ms
Wall time: 38 ms


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr,"Distance_to_(0, 0)","Within_15000000_m_from_(0, 0)",Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr
DateTime,traj_id,,,,,,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN,1.221587e+07,True,NaN,NaN,NaN
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153,1.221587e+07,True,13.690153,NaN,NaN
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428,1.221588e+07,True,0.000158,-0.000292,NaN
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895,1.221588e+07,True,0.364217,0.072812,0.014621
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073,1.221588e+07,True,0.577934,0.042744,-0.006014


In [24]:
%%time

# Now lets check the jerk for the atlantic dataset.
atlantic = spatial.create_jerk_from_prev_column(atlantic)
atlantic.head()

CPU times: user 19.7 ms, sys: 0 ns, total: 19.7 ms
Wall time: 18.3 ms


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-07-05 12:00:00,AL021851,UNNAMED,18510705,1200,,HU,22.2,-97.6,80,-999,-999,...,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN
1851-07-10 12:00:00,AL031851,UNNAMED,18510710,1200,,TS,12.0,-60.0,50,-999,-999,...,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN
1853-08-05 12:00:00,AL011853,UNNAMED,18530805,1200,,TS,32.5,-69.0,50,-999,-999,...,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN
1853-08-10 12:00:00,AL021853,UNNAMED,18530810,1200,,TS,12.0,-60.0,40,-999,-999,...,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN
1853-09-21 12:00:00,AL051853,UNNAMED,18530921,1200,,TS,20.0,-95.0,50,-999,-999,...,-999,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN


In [25]:
%%time

# Now, lets check the jerk for the seagulls dataset.
gulls = spatial.create_jerk_from_prev_column(gulls)
gulls.head()

CPU times: user 24.1 ms, sys: 12 µs, total: 24.1 ms
Wall time: 22.6 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr
DateTime,traj_id,,,,,,,,,,,,,
2009-08-15 15:00:00,91733,1082636186,True,7.91883,54.18600,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,NaN,NaN,NaN,NaN,NaN
2009-08-15 21:00:00,91733,1082636187,True,7.91917,54.18567,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,42.847231,42.847231,0.001984,NaN,NaN
2009-08-16 06:00:00,91733,1082636188,True,7.91500,54.18533,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,273.951247,260.103940,0.008455,1.997413e-07,NaN
2009-08-16 09:00:00,91733,1082636189,True,7.91917,54.18583,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,276.967082,29.098770,0.025645,1.591650e-06,1.288804e-10
2009-08-16 15:00:00,91733,1082636190,True,8.64117,54.33383,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,49697.545831,49712.038359,2.300812,1.053318e-04,4.802785e-09


In [26]:
%%time

# Now lets check the bearing between 2 consecutive
# points.
# First, we will do it for the geolife dataset.
geolife = spatial.create_bearing_column(geolife)
geolife.head()

CPU times: user 393 ms, sys: 111 ms, total: 504 ms
Wall time: 1.04 s


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr,"Distance_to_(0, 0)","Within_15000000_m_from_(0, 0)",Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive
DateTime,traj_id,,,,,,,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN,1.221587e+07,True,NaN,NaN,NaN,NaN
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153,1.221587e+07,True,13.690153,NaN,NaN,32.358601
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428,1.221588e+07,True,0.000158,-0.000292,NaN,67.015294
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895,1.221588e+07,True,0.364217,0.072812,0.014621,-142.539885
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073,1.221588e+07,True,0.577934,0.042744,-0.006014,76.651056


In [27]:
%%time

# Now lets check the bearing for the atlantic dataset.
atlantic = spatial.create_bearing_column(atlantic)
atlantic.head()

CPU times: user 207 ms, sys: 185 ms, total: 392 ms
Wall time: 417 ms


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,High Wind NE,High Wind SE,High Wind SW,High Wind NW,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-07-05 12:00:00,AL021851,UNNAMED,18510705,1200,,HU,22.2,-97.6,80,-999,-999,...,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN
1851-07-10 12:00:00,AL031851,UNNAMED,18510710,1200,,TS,12.0,-60.0,50,-999,-999,...,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN
1853-08-05 12:00:00,AL011853,UNNAMED,18530805,1200,,TS,32.5,-69.0,50,-999,-999,...,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN
1853-08-10 12:00:00,AL021853,UNNAMED,18530810,1200,,TS,12.0,-60.0,40,-999,-999,...,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN
1853-09-21 12:00:00,AL051853,UNNAMED,18530921,1200,,TS,20.0,-95.0,50,-999,-999,...,-999,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
%%time

# Now lets check the bearing for the seagulls dataset.
gulls = spatial.create_bearing_column(gulls)
gulls.head()


CPU times: user 238 ms, sys: 57.8 ms, total: 296 ms
Wall time: 596 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive
DateTime,traj_id,,,,,,,,,,,,,,
2009-08-15 15:00:00,91733,1082636186,True,7.91883,54.18600,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,NaN,NaN,NaN,NaN,NaN,NaN
2009-08-15 21:00:00,91733,1082636187,True,7.91917,54.18567,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,42.847231,42.847231,0.001984,NaN,NaN,148.914497
2009-08-16 06:00:00,91733,1082636188,True,7.91500,54.18533,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,273.951247,260.103940,0.008455,1.997413e-07,NaN,-97.930652
2009-08-16 09:00:00,91733,1082636189,True,7.91917,54.18583,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,276.967082,29.098770,0.025645,1.591650e-06,1.288804e-10,78.418266
2009-08-16 15:00:00,91733,1082636190,True,8.64117,54.33383,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,49697.545831,49712.038359,2.300812,1.053318e-04,4.802785e-09,70.369350


In [29]:
%%time

# Now, lets calculate the bearing rate between 2 consecutive points.
# First, we will do it for the geolife dataset.
geolife = spatial.create_bearing_rate_column(geolife)
geolife.head()

CPU times: user 43 ms, sys: 3.81 ms, total: 46.8 ms
Wall time: 47.7 ms


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr,"Distance_to_(0, 0)","Within_15000000_m_from_(0, 0)",Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive,Bearing_rate_from_prev
DateTime,traj_id,,,,,,,,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN,1.221587e+07,True,NaN,NaN,NaN,NaN,NaN
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153,1.221587e+07,True,13.690153,NaN,NaN,32.358601,NaN
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428,1.221588e+07,True,0.000158,-0.000292,NaN,67.015294,0.000740
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895,1.221588e+07,True,0.364217,0.072812,0.014621,-142.539885,-41.911036
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073,1.221588e+07,True,0.577934,0.042744,-0.006014,76.651056,43.838188


In [30]:
%%time

# Now lets calculate the bearing rate for the atlantic dataset.
atlantic = spatial.create_bearing_rate_column(atlantic)
atlantic.head()

CPU times: user 16.6 ms, sys: 106 µs, total: 16.7 ms
Wall time: 15.5 ms


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,High Wind SE,High Wind SW,High Wind NW,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive,Bearing_rate_from_prev
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-07-05 12:00:00,AL021851,UNNAMED,18510705,1200,,HU,22.2,-97.6,80,-999,-999,...,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1851-07-10 12:00:00,AL031851,UNNAMED,18510710,1200,,TS,12.0,-60.0,50,-999,-999,...,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853-08-05 12:00:00,AL011853,UNNAMED,18530805,1200,,TS,32.5,-69.0,50,-999,-999,...,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853-08-10 12:00:00,AL021853,UNNAMED,18530810,1200,,TS,12.0,-60.0,40,-999,-999,...,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853-09-21 12:00:00,AL051853,UNNAMED,18530921,1200,,TS,20.0,-95.0,50,-999,-999,...,-999,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
%%time

# Now lets calculate the bearing rate for the seagulls dataset.
gulls = spatial.create_bearing_rate_column(gulls)
gulls.head()

CPU times: user 28.6 ms, sys: 96 µs, total: 28.7 ms
Wall time: 28.2 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive,Bearing_rate_from_prev
DateTime,traj_id,,,,,,,,,,,,,,,
2009-08-15 15:00:00,91733,1082636186,True,7.91883,54.18600,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-08-15 21:00:00,91733,1082636187,True,7.91917,54.18567,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,42.847231,42.847231,0.001984,NaN,NaN,148.914497,NaN
2009-08-16 06:00:00,91733,1082636188,True,7.91500,54.18533,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,273.951247,260.103940,0.008455,1.997413e-07,NaN,-97.930652,-0.007619
2009-08-16 09:00:00,91733,1082636189,True,7.91917,54.18583,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,276.967082,29.098770,0.025645,1.591650e-06,1.288804e-10,78.418266,0.016329
2009-08-16 15:00:00,91733,1082636190,True,8.64117,54.33383,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,49697.545831,49712.038359,2.300812,1.053318e-04,4.802785e-09,70.369350,-0.000373


In [32]:
%%time

# Now, lets calculate the rate of bearing rate between 2
# consecutive points.
geolife = spatial.create_rate_of_bearing_rate_column(geolife)
geolife.head()


CPU times: user 31.3 ms, sys: 3.18 ms, total: 34.5 ms
Wall time: 33 ms


,,lat,lon,Distance_prev_to_curr,Distance_start_to_curr,"Distance_to_(0, 0)","Within_15000000_m_from_(0, 0)",Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive,Bearing_rate_from_prev,Rate_of_bearing_rate_from_prev
DateTime,traj_id,,,,,,,,,,,,
2008-10-23 16:53:05,1,39.984094,116.319236,NaN,NaN,1.221587e+07,True,NaN,NaN,NaN,NaN,NaN,NaN
2008-10-23 16:53:06,1,39.984198,116.319322,13.690153,13.690153,1.221587e+07,True,13.690153,NaN,NaN,32.358601,NaN,NaN
2008-10-23 05:53:11,1,39.984224,116.319402,7.403788,20.223428,1.221588e+07,True,0.000158,-0.000292,NaN,67.015294,0.000740,NaN
2008-10-23 05:53:16,1,39.984211,116.319389,1.821083,18.416895,1.221588e+07,True,0.364217,0.072812,0.014621,-142.539885,-41.911036,-8.382355
2008-10-23 05:53:21,1,39.984217,116.319422,2.889671,20.933073,1.221588e+07,True,0.577934,0.042744,-0.006014,76.651056,43.838188,17.149845


In [33]:
%%time

# Now, lets calculate the rate of bearing rate for the atlantic
# dataset.
atlantic = spatial.create_rate_of_bearing_rate_column(atlantic)
atlantic.head()

CPU times: user 20.3 ms, sys: 50 µs, total: 20.4 ms
Wall time: 19 ms


,,Name,Date,Time,Event,Status,lat,lon,Maximum Wind,Minimum Pressure,Low Wind NE,...,High Wind SW,High Wind NW,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive,Bearing_rate_from_prev,Rate_of_bearing_rate_from_prev
DateTime,traj_id,,,,,,,,,,,,,,,,,,,,,
1851-07-05 12:00:00,AL021851,UNNAMED,18510705,1200,,HU,22.2,-97.6,80,-999,-999,...,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1851-07-10 12:00:00,AL031851,UNNAMED,18510710,1200,,TS,12.0,-60.0,50,-999,-999,...,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853-08-05 12:00:00,AL011853,UNNAMED,18530805,1200,,TS,32.5,-69.0,50,-999,-999,...,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853-08-10 12:00:00,AL021853,UNNAMED,18530810,1200,,TS,12.0,-60.0,40,-999,-999,...,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1853-09-21 12:00:00,AL051853,UNNAMED,18530921,1200,,TS,20.0,-95.0,50,-999,-999,...,-999,-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
%%time

# Now lets calculate the rate of bearing rate for the seagulls
# dataset.
gulls = spatial.create_rate_of_bearing_rate_column(gulls)
gulls.head()

CPU times: user 21.8 ms, sys: 0 ns, total: 21.8 ms
Wall time: 19.6 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Distance_prev_to_curr,Distance_start_to_curr,Speed_prev_to_curr,Acceleration_prev_to_curr,jerk_prev_to_curr,Bearing_between_consecutive,Bearing_rate_from_prev,Rate_of_bearing_rate_from_prev
DateTime,traj_id,,,,,,,,,,,,,,,,
2009-08-15 15:00:00,91733,1082636186,True,7.91883,54.18600,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-08-15 21:00:00,91733,1082636187,True,7.91917,54.18567,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,42.847231,42.847231,0.001984,NaN,NaN,148.914497,NaN,NaN
2009-08-16 06:00:00,91733,1082636188,True,7.91500,54.18533,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,273.951247,260.103940,0.008455,1.997413e-07,NaN,-97.930652,-0.007619,NaN
2009-08-16 09:00:00,91733,1082636189,True,7.91917,54.18583,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,276.967082,29.098770,0.025645,1.591650e-06,1.288804e-10,78.418266,0.016329,2.217341e-06
2009-08-16 15:00:00,91733,1082636190,True,8.64117,54.33383,gps,Larus fuscus,91733A,Navigation experiments in lesser black-backed ...,49697.545831,49712.038359,2.300812,1.053318e-04,4.802785e-09,70.369350,-0.000373,-7.732055e-07


In [35]:
%%time

# Now, lets get the start locations of all the trajectories
# in the atlantic dataset.
spatial.get_start_location(atlantic)

CPU times: user 577 ms, sys: 530 ms, total: 1.11 s
Wall time: 4.24 s


,lat,lon
traj_id,,
AL011853,32.5,-69.0
AL051867,17.0,-58.0
AL021853,12.0,-60.0
AL041862,13.0,-61.0
AL031867,14.0,-77.0
...,...,...
AL041957,13.0,-21.3
AL142012,15.5,-38.0
AL201969,16.0,-47.0


In [36]:
%%time

# Now, lets get the end location of a specific trajectory
# from the atlantic dataset.
spatial.get_end_location(atlantic, 'AL011851')

CPU times: user 37.9 ms, sys: 8.39 ms, total: 46.3 ms
Wall time: 45.2 ms


(31.0, -100.2)

In [37]:
%%time

# Now, lets get the distance travelled by a specific trajectory
# in the atlantic dataset.
spatial.get_distance_travelled_by_traj_id(atlantic, 'AL011851')

CPU times: user 42.8 ms, sys: 13.2 ms, total: 56 ms
Wall time: 65.1 ms


716547.106240304

In [38]:
%%time

# Now, lets get the number of unique locations visited by
# all the trajectories in the atlantic dataset.
spatial.get_number_of_locations(atlantic)

CPU times: user 482 ms, sys: 499 ms, total: 981 ms
Wall time: 3.28 s


,Number of Unique Coordinates
traj_id,
AL011853,1
AL051867,1
AL021853,1
AL041862,1
AL031867,1
...,...
AL041957,95
AL142012,96
AL201969,99


In [39]:
end = time.time()
print(f"Total execution time is {end - start} seconds.")

Total execution time is 35.47942900657654 seconds.
